In [1]:
import cv2
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from IPython.display import display, clear_output
from PIL import Image
import numpy as np

In [2]:
# Load YOLOv8 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 

Using cache found in /Users/achbj/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-11-22 Python-3.10.11 torch-2.1.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
# Initialize face recognition model
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [4]:
# Load the target face and compute embeddings
target_image = cv2.imread('./data/om.png')
target_cropped = mtcnn(target_image)
if target_cropped is not None:
    target_embedding = resnet(target_cropped.unsqueeze(0))

# Open video
cap = cv2.VideoCapture('./data/ytv.mp4')
processed_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces using YOLOv5
    results = model(frame)
    threshold = 1
    # Process each detected face
    for detection in results.xyxy[0]:
        # Extract face
        x1, y1, x2, y2 = map(int, detection[:4])
        face = frame[y1:y2, x1:x2]

        # Compute embedding and compare
        cropped_face = mtcnn(face)
        if cropped_face is not None:
            face_embedding = resnet(cropped_face.unsqueeze(0))
            if torch.dist(face_embedding, target_embedding) < threshold:  # Now using the defined threshold
                # Draw a bounding box and label for recognized face
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, 'Recognized', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Convert the frame to RGB and store in the list
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    processed_frames.append(frame_rgb)

    

# Release video
cap.release()

# Display all processed frames
for frame_rgb in processed_frames:
    frame_pil = Image.fromarray(frame_rgb)
    clear_output(wait=True)
    display(frame_pil)
    # Adjust the time delay as needed for frame rate control
    time.sleep(0.033)  # Delay for ~30fps. Adjust as needed.

cv2.destroyAllWindows()

[ WARN:0@14.377] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@14.377] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@14.377] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@14.377] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/s

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
# Read the image
image = cv2.imread('./data/Screenshot 2023-11-22 at 6.33.12 PM.png')  # Replace with your image path

# Display the image in a window
cv2.imshow('Image Window', image)

# Wait for a key press to close the window
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()